### This Notebook scrapes all tracks and saves the pos data into a parquet-file

In [2]:
import fastf1 as ff1
import numpy as np
import pandas as pd
from tqdm import tqdm

Get all GPs from the years 2018 to 2024

In [3]:
all_gps = []
for year in range(2020,2025):
    temp_df = pd.DataFrame(ff1.get_event_schedule(year)["RoundNumber"])
    temp_df["year"] = year
    all_gps.append(temp_df)




df_all_gps = pd.concat(all_gps)

req         WARNING 	DEFAULT CACHE ENABLED! (16.25 GB) /Users/max/Library/Caches/fastf1


Drop testing events

In [4]:
df_all_gps = df_all_gps.replace({0:None},regex=True)
df_all_gps = df_all_gps.dropna()

Scrape the track data for every track

In [5]:
# Hilfsfunktion um Positionen gleich richtig zu rotieren
def rotate(xy, *, angle):
    rot_mat = np.array(
        [[np.cos(angle), np.sin(angle)], [-np.sin(angle), np.cos(angle)]]
    )
    return np.matmul(xy, rot_mat)

In [6]:
gp_data

NameError: name 'gp_data' is not defined

In [7]:
all_track_data = []
for _,gp_data in tqdm(df_all_gps.iterrows()):
    year = gp_data["year"]
    gp = gp_data["RoundNumber"]

    session = ff1.get_session(year,gp,"Q")
    session.load()
    df_temp_pos_data = session.laps.pick_fastest().get_pos_data()
    # add the first row to end to make the lap complete
    temp_first_row = df_temp_pos_data.iloc[:1,:]
    df_temp_pos_data = pd.concat([df_temp_pos_data,temp_first_row])
    
    track = df_temp_pos_data.loc[:, ("X", "Y")].to_numpy()

    circuit_info = session.get_circuit_info()
    # Convert the rotation angle from degrees to radian.
    track_angle = circuit_info.rotation / 180 * np.pi

    # Rotate and plot the track map.
    rotated_track = rotate(track, angle=track_angle)
    
    df_temp_track_data = pd.DataFrame(rotated_track, columns=["x", "y"])
    
    df_temp_track_data["event_name"] = gp
    df_temp_track_data["year"] = year
    all_track_data.append(df_temp_track_data)

df_track_data = pd.concat(all_track_data)



0it [00:00, ?it/s]

core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '4', '23', '11', '16', '55', '18', '3', '5', '10', '26', '31', '8', '20', '63', '99', '7', '6']
1it [00:01,  1.94s/it]core           INFO 	Loading data for Styrian Grand Prix - Qualifying [v3.3.7]
req      

KeyError: 'DriverNumber'

In [9]:
df_track_data = pd.concat(all_track_data)


Save to parquet

In [10]:
df_track_data

,x,y,event_name,year
0,1001.509178,-1223.707631,1,2020
1,825.373851,-1274.789397,1,2020
2,646.238981,-1325.923519,1,2020
3,467.121564,-1378.057489,1,2020
4,287.004299,-1430.208912,1,2020
...,...,...,...,...
264,-10471.544323,299.106168,8,2024
265,-10386.691509,405.172186,8,2024
266,-10229.713803,587.605735,8,2024
267,-10098.899049,736.805266,8,2024


In [11]:
df_track_data.to_parquet("./static/data/all_tracks.parquet",index=False)